In [ ]:
!pip install ktrain

     |████████████████████████████████| 25.3MB 127kB/s 
     |████████████████████████████████| 983kB 48.0MB/s 
     |████████████████████████████████| 266kB 44.5MB/s 
     |████████████████████████████████| 1.3MB 49.5MB/s 
     |████████████████████████████████| 471kB 45.3MB/s 
     |████████████████████████████████| 890kB 45.2MB/s 
     |████████████████████████████████| 1.1MB 39.3MB/s 
     |████████████████████████████████| 2.9MB 37.3MB/s 
  Created wheel for ktrain: filename=ktrain-0.23.2-cp36-none-any.whl size=25272006 sha256=5c2b23b605bbdd4fe086bde55bfc411837fffbed740e43d7baf1ae1032a85d19
  Stored in directory: /root/.cache/pip/wheels/fe/4a/b1/aff404c4e0893ca1c0b64781d0298b7ff6de94d5117a0e7d5c
  Created wheel for keras-bert: filename=keras_bert-0.86.0-cp36-none-any.whl size=34145 sha256=698cd73008a20ca35652b9df0158dcb3e7526dce93793b049e3a730f39fbc8ba
  Stored in directory: /root/.cache/pip/wheels/66/f0/b1/748128b58562fc9e31b907bb5e2ab6a35eb37695e83911236b
  Created wheel for lan

In [ ]:
import tensorflow as tf
tf.__version__

'2.3.0'

In [ ]:
!git clone https://github.com/laxmimerit/IMDB-Movie-Reviews-Large-Dataset-50k.git


Cloning into 'IMDB-Movie-Reviews-Large-Dataset-50k'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 10 (delta 1), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (10/10), done.


In [ ]:
import pandas as pd
import numpy as np
import ktrain
from ktrain import text
import tensorflow as tf


In [ ]:
train_data = pd.read_excel('/content/IMDB-Movie-Reviews-Large-Dataset-50k/train.xlsx',dtype=str)

In [ ]:
test_data = pd.read_excel('/content/IMDB-Movie-Reviews-Large-Dataset-50k/test.xlsx',dtype=str)

In [ ]:
train_data.head()

,Reviews,Sentiment
0,"When I first tuned in on this morning news, I ...",neg
1,"Mere thoughts of ""Going Overboard"" (aka ""Babes...",neg
2,Why does this movie fall WELL below standards?...,neg
3,Wow and I thought that any Steven Segal movie ...,neg
4,"The story is seen before, but that does'n matt...",neg


In [ ]:
train_data.tail()

,Reviews,Sentiment
24995,Everyone plays their part pretty well in this ...,pos
24996,It happened with Assault on Prescient 13 in 20...,neg
24997,My God. This movie was awful. I can't complain...,neg
24998,"When I first popped in Happy Birthday to Me, I...",neg
24999,"So why does this show suck? Unfortunately, tha...",neg


In [ ]:
train_data.shape,test_data.shape

((25000, 2), (25000, 2))

In [ ]:
(X_train,y_train),(X_test,y_test),preproc= text.texts_from_df(train_df=train_data,
                                                      text_column = 'Reviews',
                                                      label_columns = 'Sentiment',
                                                      val_df = test_data,
                                                      maxlen = 500,
                                                      preprocess_mode = 'bert'
                                                      )

downloading pretrained BERT model (uncased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


In [ ]:
model = text.text_classifier(name='bert',
                             train_data=(X_train,y_train),
                             preproc=preproc)

Is Multi-Label? False
maxlen is 500
done.


In [ ]:
learner = ktrain.get_learner(model=model,train_data=(X_train,y_train),val_data=(X_test,y_test),batch_size=6)
learner

In [ ]:
# find out best larning rate
#learner.lr_find()
#learner.lr_plot
#it takes many days to find out

In [ ]:
learner.fit_onecycle(lr=2e-5, epochs=1)



begin training using onecycle policy with max lr of 2e-05...
4167/4167 [==============================] - 5425s 1s/step - loss: 0.2480 - accuracy: 0.8972 - val_loss: 0.0992 - val_accuracy: 0.9684


In [ ]:
predictor = ktrain.get_predictor(learner.model,preproc)

In [ ]:
data = ['this movie was not really terrible, actors acting was ok, director was done good job','movie was ok, direction was not upto the mark, plot was good',
        'what a plot, great acting, direction was excellent']

In [ ]:
predictor.predict(data)

['neg', 'neg', 'pos']

In [40]:
predictor.predict(data,return_proba=True)

array([[0.76734155, 0.23265843],
       [0.8544667 , 0.1455333 ],
       [0.00432319, 0.9956768 ]], dtype=float32)

In [41]:
# save the model
predictor.save('/content/bert')

In [42]:
!zip -r /content/bert.zip /content/bert

  adding: content/bert/ (stored 0%)
  adding: content/bert/tf_model.preproc (deflated 52%)
  adding: content/bert/.zip (stored 0%)
  adding: content/bert/tf_model.h5 (deflated 11%)


In [43]:
predictor_load = ktrain.load_predictor('/content/bert')

In [46]:
predictor_load.get_classes()

['neg', 'pos']

In [47]:
predictor_load.predict(data)

['neg', 'neg', 'pos']